# Run Any Kind of OLS Regression (ANOVA, GLM, etc.)

### Authors: Calvin Howard.

#### Last updated: July 6, 2023

Use this to run/test a statistical model (e.g., regression or T-tests) on a spreadsheet.

Notes:
- To best use this notebook, you should be familar with GLM design and Contrast Matrix design. See this webpage to get started:
[FSL's GLM page](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/GLM)

# 00 - Import CSV with All Data
**The CSV is expected to be in this format**
- ID and absolute paths to niftis are critical
```
+-----+----------------------------+--------------+--------------+--------------+
| ID  | Nifti_File_Path            | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

Prep Output Direction

In [1]:
# Specify where you want to save your results to
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/atrophy_seeds_2023/Figures/regression_to_baseline/r_squared_plots'

Import Data

In [2]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/cognition_2023/metadata/master_list_proper_subjects.xlsx'
sheet = 'master_list_proper_subjects'

In [ ]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()


# 01 - Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [ ]:
data_df.columns

In [5]:
drop_list = ['Z_Scored_Percent_Cognitive_Improvement', 'Z_Scored_Subiculum_Connectivity_T', 'Age']

In [ ]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)
data_df

**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

In [ ]:
data_df.columns

Set the parameters for dropping rows

In [8]:
column = 'Cohort'  # The column you'd like to evaluate
condition = 'equal'  # The condition to check ('equal', 'above', 'below', 'not')
value = 'Queensland' # The value to drop if T

In [ ]:
data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
display(data_df)

**Standardize Data**
- Enter Columns you Don't want to standardize into a list

In [ ]:
data_df.columns

In [11]:
# Remove anything you don't want to standardize
# cols_not_to_standardize = ['subid'] #['Age']

In [ ]:
# data_df = cal_palm.standardize_columns(cols_not_to_standardize)
data_df

In [ ]:
data_df.columns

Regress out Covariate

In [14]:
# from calvin_utils.statistical_utils.regression_utils import RegressOutCovariates
# # use this code block to regress out covariates. Generally better to just include as covariates in a model..
# dependent_variable_list = lis
# regressors = ['Age', 'Sex']

# data_df, adjusted_dep_vars_list = RegressOutCovariates.run(df=data_df, dependent_variable_list=dependent_variable_list, covariates_list=regressors)
# print(adjusted_dep_vars_list)

Edit Column Values

In [ ]:
data_df.columns

In [16]:
import pandas as pd
import numpy as np
def convert_to_ordinal(data_df, columns):
    """
    Convert unique values in specified columns of a DataFrame to ordinal values and print the mapping.

    Parameters:
    - data_df (pd.DataFrame): DataFrame containing the data to be converted.
    - columns (list): List of column names to be converted to ordinal values.

    Returns:
    - ordinal_df (pd.DataFrame): DataFrame with specified columns converted to ordinal values.
    - mapping_dict (dict): Dictionary showing the mapping of original values to ordinal values for each column.
    """
    ordinal_df = data_df.copy()
    mapping_dict = {}

    for column in columns:
        if column in ordinal_df.columns:
            unique_values = ordinal_df[column].unique()
            unique_values.sort()  # Ensure the values are sorted before assigning ordinals
            mapping_dict[column] = {value: idx for idx, value in enumerate(unique_values)}
            ordinal_df[column] = ordinal_df[column].map(mapping_dict[column])

    print("Mapping of unique values to ordinal values:")
    for column, mapping in mapping_dict.items():
        print(f"{column}: {mapping}")

    return ordinal_df

# F-Test of Variances

In [ ]:
data_df.columns

In [ ]:
data_df.columns

In [ ]:
from calvin_utils.statistical_utils.f_test_of_variance import orchestrate_f_test
orchestrate_f_test(df=data_df, dv_col='Z_Scored_Percent_Cognitive_Improvement', iv_col='StimMatch', 
                   comparison_value=0, categorical_col=None)